***Importing Libraries***

In [15]:
from sklearn.preprocessing import LabelEncoder

#for the NN
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
import tensorflow as tf

#for inbalance issues 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from collections import Counter
from matplotlib import pyplot 
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import NearMiss
from imblearn.combine import SMOTEENN

In [2]:
#SMOTE for dealing with unbalanced data
#pip install imblearn
########################################################### different methods to balance data
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

def smote(K,x,y):
    seed=100 
    sm=SMOTE(sampling_strategy='auto', k_neighbors=K, random_state=seed)
    #sm=SMOTE()
    X_res, y_res=sm.fit_resample(x,y)

    #print("SMOTE\n# of 1 %d\n# of 0 %d"%(np.count_nonzero(y_res==1),np.count_nonzero(y_res==0)))
    return X_res, y_res

def ada(x,y):
    adas=ADASYN()
    c_res,d_res=adas.fit_resample(x,y)
    #print("ADASYN\n# of 1 %d\n# of 0 %d"%(np.count_nonzero(d_res==1),np.count_nonzero(d_res==0)))
    return c_res, d_res

def ENN(K, x,y):
    undersample= EditedNearestNeighbours(n_neighbors=K)
    a,b=undersample.fit_resample(x,y)
    return a, b

def CNN(K,x,y):#relativly slow so do not use again 
    undersample=CondensedNearestNeighbour(n_neighbors=K)
    X,Y=undersample.fit_resample(x,y)
    return X,Y

def OSS(n,s,x,y):# possibly slow because it uses CNN
    undersample=OneSidedSelection(n_neighbors=n, n_seeds_S=s)
    X,Y=undersample.fit_resample(x,y)
    return X,Y

def NM(n,x,y):# DO NOT USE v 2 TOOK UP 54G of RAM
    undersample=NearMiss(version=3,n_neighbors_ver3=n)
    X,Y=undersample.fit_resample(x,y)
    return X,Y
def comb(x,y):
    #resample = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))
    resample = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
    retx,rety=resample.fit_resample(x,y)
    return retx, rety

In [23]:
import pandas as pd
import numpy as np
from ReadDataset import readFiles


def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            if str(col_type)[:3] == 'int':
                if df[col].max() < 2 ** 7:
                    df[col] = df[col].astype(np.int8)
                elif df[col].max() < 2 ** 15:
                    df[col] = df[col].astype(np.int16)
                elif df[col].max() < 2 ** 31:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)  
            else:
                if df[col].max() < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif df[col].max() < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def simplifyColumnValues(df):
    df.loc[df['DeviceInfo'].str.contains('SM', na=False), 'DeviceInfo'] = 'Samsung'
    df.loc[df['DeviceInfo'].str.contains('SAMSUNG', na=False), 'DeviceInfo'] = 'Samsung'
    df.loc[df['DeviceInfo'].str.contains('Moto', na=False), 'DeviceInfo'] = 'Motorola'
    df.loc[df['DeviceInfo'].str.contains('moto', na=False), 'DeviceInfo'] = 'Motorola'
    df.loc[df['DeviceInfo'].str.contains('LG-', na=False), 'DeviceInfo'] = 'LG'
    df.loc[df['DeviceInfo'].str.contains('rv:', na=False), 'DeviceInfo'] = 'RV'
    df.loc[df['DeviceInfo'].str.contains('HUAWEI', na=False), 'DeviceInfo'] = 'Huawei'
    df.loc[df['DeviceInfo'].str.contains('Blade', na=False), 'DeviceInfo'] = 'ZTE'
    df.loc[df['DeviceInfo'].str.contains('BLADE', na=False), 'DeviceInfo'] = 'ZTE'
    df.loc[df['DeviceInfo'].str.contains('Linux', na=False), 'DeviceInfo'] = 'Linux'
    df.loc[df['DeviceInfo'].str.contains('ASUS', na=False), 'DeviceInfo'] = 'Asus'
    return df

def generateNewFeatures(train, test):
    # New feature - decimal part of the transaction amount.
    train['TransactionAmt_decimal'] = ((train['TransactionAmt'] - train['TransactionAmt'].astype(int)) * 1000).astype(int)
    test['TransactionAmt_decimal'] = ((test['TransactionAmt'] - test['TransactionAmt'].astype(int)) * 1000).astype(int)

    # New feature - day of week in which a transaction happened.
    train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
    test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)

    # New feature - hour of the day in which a transaction happened.
    train['Transaction_hour'] = np.floor(train['TransactionDT'] / 3600) % 24
    test['Transaction_hour'] = np.floor(test['TransactionDT'] / 3600) % 24

    return train, test

# simply replace missing values using -9999
# don't want to use techniques like mean, median, and mode as they will make the fraud transaction too similar to the non-fraud transactions
def replaceMissingValues(df):
    for col in df.columns:
        if df[col].dtype.name=='category':
            df[col]=df[col].add_categories("-x")
            df[col].fillna('-x')
        else:
            df[col].fillna(-9999)
    return df

# dimensionality reduction: removing features with less importance
def removeFeatures(train, test):
    # useful features extracted from Principal Component Analysis
    useful_features = ['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                       'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                       'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                       'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                       'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                       'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                       'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                       'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                       'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                       'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                       'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                       'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                       'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                       'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                       'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                       'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294', 'V303',
                       'V304', 'V306', 'V307', 'V308', 'V310', 'V312', 'V313', 'V314', 'V315', 'V317', 'V322', 'V323', 'V324', 'V326',
                       'V329', 'V331', 'V332', 'V333', 'V335', 'V336', 'V338', 'id_01', 'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                       'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                       'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'device_name', 'device_version', 'OS_id_30', 'version_id_30',
                       'browser_id_31', 'version_id_31', 'screen_width', 'screen_height', 'had_id']
    
    # determining columns not found in useful features list
    cols_to_drop = [col for col in train.columns if col not in useful_features]
    cols_to_drop.remove('isFraud')
    cols_to_drop.remove('TransactionDT')

    # removing columns from both train and test
    train = train.drop(cols_to_drop, axis=1)
    test = test.drop(cols_to_drop, axis=1)
    
    return train, test

def makeNumeric(dat):
    enc=LabelEncoder()# create the label encoder
    for cols in dat:#loop through all of the columns
        if dat[cols].dtype.name=="object":#if the data is in a string format we will need to convert it to numeric to find the correlation 
            enc.fit(dat[cols].astype(str))#fit the column to the encoder to convert to numeric 
            dat[cols]=enc.fit_transform(dat[cols].astype(str))#transform the data into numeric 
    return dat

def label_encode_categorical_features(data):
    encoder_dict = collections.defaultdict(LabelEncoder)
    for cols in data:
        if data[cols].dtype.name=="object":
            data[cols] = data[cols].apply(lambda x: encoder_dict[x.name].fit_transform(x))
    return data

In [25]:
train_identity = pd.read_csv("D:\School\Fifth Year\Large Scale Data Analytics\Project\ieee-fraud-detection\\train_identity.csv")
train_transaction = pd.read_csv("D:\School\Fifth Year\Large Scale Data Analytics\Project\ieee-fraud-detection\\train_transaction.csv")

full=train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
full=replaceMissingValues(full)
full=pd.DataFrame(data=full)
full=reduce_mem_usage(full)
full=simplifyColumnValues(full)



Memory usage of dataframe is 1959.88 MB
Memory usage after optimization is: 527.82 MB
Decreased by 73.1%


ValueError: cannot index with vector containing NA / NaN values

In [ ]:
#Keras example for getting the NN to work 
from keras.optimizers import SGD, RMSprop
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc, roc_auc_score
sgd=SGD(lr=0.1)
def NNetwork(Tr,A,E,L1,L2,L3,L4,W):#Tr is training set A is answer L1 is layer 1 nodes L2 is layer 2 nodes E is the epochs and W is if weights are needed
    NNIn=Tr#take the data and use  new name for it 
    NNans=A
    ##########################################layers are added and removed here to test different implementations
    model_NN=Sequential()
    model_NN.add(Dense(L1, input_dim=NNIn.shape[1], activation='relu'))#50 nodes in first hidden layer there might have been an issue with relu but it seems to be fixed
    model_NN.add(Dropout(0.1))
    model_NN.add(Dense(L2, activation='relu'))
    #model_NN.add(Dense(L3, activation='relu',kernel_initializer='uniform'))
    
    #model_NN.add(Dense(L3, activation='relu'))#seems to bring back the all same value issue 
    #model_NN.add(Dropout(0.1))
    model_NN.add(Dense(1, activation='sigmoid'))
    model_NN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model_NN.compile(optimizer=sgd,loss='mse')##################################################try another optimizer
    if(W==1):#if the weights are needed or not 
        weights={0:(np.count_nonzero(NNans==1)/NNans.size), 1:(np.count_nonzero(NNans==0)/NNans.size)}#gives more even distribution less loss and not much difference in accuracy 
        #weights={0:7, 1:193}#increase to 27 gave slightly better results 
        model_NN.fit(NNIn,NNans,class_weight=weights,epochs=E)# it says 11073 insteead of 354324 because there are 32 batches and 354324/32=11073
        
    else: 
        model_NN.fit(NNIn,NNans,epochs=E)
    
   
    return model_NN
    NNacc(model_NN,NNIn,NNans)#call the other function to analyze 

    
def NNacc (model, testin,testout):
    predictions=model.predict_classes(testin)
    check=np.array(testout)
    count=0
    count1=0
    count0=0
    for i in range(predictions.size):
        if(predictions[i]==check[i]):
            count+=1
        if( predictions[i]==1 and check[i]==1):
            count1+=1
        if( predictions[i]==0 and check[i]==0):
            count0+=1 
    acc=((count/testout.size))*100
    acc1=(count1/np.count_nonzero(check==1))*100
    acc0=(count0/np.count_nonzero(check==0))*100
    print("Total accuracy: %d    Accuracy of 1: %d     Accuracy of 0: %d"%(round(acc),round(acc1),round(acc0)))
    print("AUC %.4f"%(roc_auc_score(testout,predictions)))
    #print(precision_score(testin,predictions.round()))
    #print(recall_score(testin,predictions.round()))
    #print(f1_score(testin,predictions.round()))
########################################### collection of different function calls to balance the data  
#bal_x, bal_y =smote(3,X_train,Y_train)#get more balanced data using smote with the input and output as well as the K value 

#ad_x,ad_y=ada(X_train,Y_train)#get more balanced data using ADASYN

#u_x,u_y=ENN(12,X_train,Y_train)

#CNN_x,CNN_y=CNN(1,X_train,Y_train)#relativly slow do not use again 
#print("Done ENN")#takes a fair amount of time 
#NM_x, NM_y=NM(3,X_train,Y_train)#try this one if they still take a long time 
#print("Done NM")#does not take long but does when using second type
#ONN_x,ONN_y=OSS(4,200,X_train, Y_train)
#print("Done ENN")
#combination 

#cx,cy=comb(X_train,Y_train)#########################################################################################################################UNDO ON FIRST RUN
#mod=NNetwork(X_train,Y_train,10,100,600,250,1)#train using the X_train and Y_train with epoch and first and second layers and specify 1 if weig

#####################################################different NN implimentations and printouts based on each data for comperison
""" print("SMOTE")
mod=NNetwork(bal_x,bal_y,10,500,1000,280,400,0)#train using the X_train and Y_train with epoch and first and second layers and specify 1 if weig
print("With balanced training data")
NNacc(mod, bal_x,bal_y)
print("With origional training data")
NNacc(mod, X_train,Y_train)
print("with origional testing data")
NNacc(mod, X_test, Y_test)


print("ADASYN")
mod=NNetwork(ad_x,ad_y,10,500,1000,280,400,0)#train using the X_train and Y_train with epoch and first and second layers and specify 1 if weig
print("With balanced training data")
NNacc(mod, ad_x,ad_y)
print("With origional training data")
NNacc(mod, X_train,Y_train)
print("with origional testing data")
NNacc(mod, X_test, Y_test) 
  

print("Edited NN")
mod=NNetwork(u_x,u_y,10,100,100,280,400,0)#
print("With balanced training data")
NNacc(mod, u_x,u_y)
print("With origional training data")
NNacc(mod, X_train,Y_train)
print("with origional testing data")
NNacc(mod, X_test, Y_test) 


print("One Sided NN")
mod=NNetwork(ONN_x,ONN_y,10,100,100,280,400,0)#
print("With balanced training data")
NNacc(mod, ONN_x,ONN_y)
print("With origional training data")
NNacc(mod, X_train,Y_train)
print("with origional testing data")
NNacc(mod, X_test, Y_test) 


print("Near Miss")
mod=NNetwork(NM_x,NM_y,10,100,600,300,400,0)#
print("With balanced training data")
NNacc(mod, NM_x,NM_y)
print("With origional training data")
NNacc(mod, X_train,Y_train)
print("with origional testing data")
NNacc(mod, X_test, Y_test) 
"""
##########################################################################most results tracked in NN Testing doc on google drive 
print("Combined")
mod=NNetwork(cx,cy,10,500,500,320,400,0)#
print("With balanced training data")
NNacc(mod, cx,cy)
print("With origional training data")
NNacc(mod, X_train,Y_train)
print("with origional testing data")
NNacc(mod, X_test, Y_test) 


""" print("NN Added Weights")
mod=NNetwork(X_train,Y_train,10,100,100,280,400,1)#train using the X_train and Y_train with epoch and first and second layers and specify 1 if weig
print("With balanced training data")
NNacc(mod, X_train,Y_train)
print("with origional testing data")
NNacc(mod, X_test, Y_test) """
